In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [52]:
# function to get url from  gender and year 
def boston_url(year, gender):
    return f"https://results.baa.org/{year}/?page=1&event=R&event_main_group=runner&num_results=500&pid=list&pidp=start&search%5Bsex%5D={gender}&search%5Bage_class%5D=%25"

# get list of urls 
list_of_urls = []
for page in range(2018,2024):
        url1 = boston_url(page, "W")
        url2 = boston_url(page, "M")
        list_of_urls.append(url1)
        list_of_urls.append(url2)



In [53]:
# lists for name, place, finish time, sex, and year 
names = []
place = []
time = []
sex = []
year = []

for url in list_of_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    gen_text = soup.find_all('li', class_='list-group-item')[0]
    sex_curr = gen_text.find('span', class_="sex").text.strip()
    year_curr = soup.find('span', class_='hidden-xs').text.strip()
    results = soup.find_all('li', class_= 'list-group-item row')
    for result in results:
        place_elem = result.find('div', class_='list-field type-place place-primary numeric')
        place.append(place_elem.text.strip())
        name_elem = result.find('h4', class_= 'list-field type-fullname')
        names.append(name_elem.text.strip())
        time_elem = result.find_all('div', class_='split list-field type-time')
        time.append(time_elem[1].text.strip())
        sex.append(sex_curr)
        year.append(year_curr)

In [55]:
# turn into dataframe 
boston = pd.DataFrame(list(zip(names, place, time, sex, year)), 
               columns =['Name', 'Place', 'Time', 'Sex', 'Year']) 
boston.to_csv('boston_2018_2023.csv', index = False)

In [57]:
# format is slightly different for 2017 and before
list_of_urls = []
for page in range(2012,2018):
        url1 = boston_url(page, "W")
        url2 = boston_url(page, "M")
        list_of_urls.append(url1)
        list_of_urls.append(url2)

In [58]:
data_all = pd.DataFrame() # initialize df 

for url in list_of_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # gender = soup.find('div', class_="list-info-text").find('span', class_="sex").text.strip()
    gender = re.findall("(?<=5D=)(.*)(?=&search)", url)
    year = re.findall("(?<=baa.org/)(.*)(?=\/\?page)",url)
    table = soup.find('table', class_='list-table')
    columns = []
    data = []

# Extract column names from the table header (th elements)
    header_row = table.find('thead').find('tr')
    for header_cell in header_row.find_all('th'):
        columns.append(header_cell.text.strip())

# Extract data rows from the table body (td elements)
    body = table.find('tbody')
    for row in body.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        data.append(row_data)
    df = pd.DataFrame(data, columns = columns)
    yearcol = year * len(df)
    sexcol = gender * len(df)
    df['year'] = yearcol 
    df['sex'] = sexcol

    data_all = data_all.append(df, ignore_index = True)

data_all.to_csv('boston_2012_2017.csv', index = False)

/Users/lilykoffman/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
